Mencoba fitur translate

In [1]:
import pyodbc
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime, timedelta

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

from googletrans import Translator

from FlaskApp.transform import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [3]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [4]:
df_job = pd.DataFrame(engine.execute(
            """
            SELECT Job.JobID, Job.JobTitle, FunctionPosition.FunctionPositionName, EducationLevel.EducationLevelName, City.Name AS CityName, Province.Name AS ProvinceName, Major.MajorName, Job.Description, Job.Requirement
            FROM (((((Job
            RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
            RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
            RIGHT JOIN City ON Job.CityID = City.CityID)
            RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
            RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
            WHERE JobStatus='Publish'
            """
        ))

In [93]:
def change_slangwords(slangwords, teks):
    if type(teks) == str:
        teks = teks.split(' ')
    for i in range(len(teks)):
        if teks[i] in slangwords:
            teks[i] = slangwords[teks[i]]
    return teks

def translate_teks(translator: Translator, teks):
    if teks == '':
        return ''
    result = translator.translate(teks, dest='id', src='en').text
    if result != teks:
        print(f'Translating {teks[:25]} ...')
    return result.lower()

def stemmer_words(stemmer, teks):
    if teks == '':
        return ''
    return stemmer.stem(teks)

In [ ]:
df_job.fillna('', inplace=True)

translator = Translator(service_urls=['translate.googleapis.com'])
factory = StemmerFactory()
stemmer = factory.create_stemmer()

focus_cleansing = df_job.Description.copy().map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace).apply(lambda x: translate_teks(translator, x)).apply(lambda x: stemmer_words(stemmer, x)).apply(lambda x: x.split(' '))

In [119]:
'''PRE-PROCESSING of "JOB"
1. GENERAL
    a. set index to JobID
    b. specify columns name consider the column data type (STR, INT, BOOL)
2. STR ==| CityName, ProvinceName, EducationLevelName, MajorName, DriverLicenseType, Gender, MaritalStatus, JobTitle, FunctionPositionName, Description, Requirement
    a. fillna with ''
    b. replace 'none' with ''
    c. casting to str
        -> lower
    d. cleansing
        -> JobTitle
            - translate
            - lower
        -> FunctionPositionName
            - translate
            - lower
        -> MajorName
            - translate
            - lower
        -> Description
            - clean text
            - maintain alphabet
            - remove single
            - remove more space
            - translate
                -> lower
            - stemm
            - tokenize
            - remove slangwords
            - remove stopwords
            - join
        -> Requirement
            - clean text
            - maintain alphabet
            - remove single
            - remove more space
            - translate
                -> lower
            - stemm
            - tokenize
            - remove slangwords
            - remove stopwords
            - join
    e. concatenate
        -> JobTitle - FunctionPositionName => JobTitlePosition
            - Drop FunctionPositionName
        -> Description - Requirement => DescriptionRequirement
            - Drop Requirement
3. INT ==| UsiaMax, SalaryMin, SalaryMax
    a. fillna with ''
    b. replace '' with 0
    c. casting to int
    d. find SalaryMean
    e. rename SalaryMin
    f. drop SalaryMax
4. BOOL ==| UsingGlasses
    a. fillna with ''
    b. casting to str
    c. lower
'''

'''PRE-PROCESSING of Applicant
1. GENERAL
    a. fillna with ''
2. APPLICANT
    a. Age
        -> pick_date
        -> filter_date 1958-2006
        -> get_age
        -> fillna with 0
        -> drop Dob
3. EDUCATION
    a. DateStart
        -> pick_date
        -> filter_date 1980-2023
    b. DateEnd
        -> pick_date
        -> filter_date 1980-2023
    c. select DateStart & DateEnd where DateStart and DateEnd is not None
    d. pick last education by sort, groupby, and agg
    e. drop DateStart and DateEnd
4. Experience
    a. DateFrom
        -> pick_date
        -> filter_date 1980-2023
    b. DateTo
        -> pick_date
        -> filter_date 1980-2023
    c. select DateFrom & DateTo where DateFrom and DateTo is not None
    d. substract_months
    e. 
'''

'PRE-PROCESSING of "JOB"\n1. GENERAL\n    a. set index to JobID\n    b. specify columns name consider the column data type (STR, INT, BOOL)\n2. STR ==| CityName, ProvinceName, EducationLevelName, MajorName, DriverLicenseType, Gender, MaritalStatus, JobTitle, FunctionPositionName, Description, Requirement\n    a. fillna with \'\'\n    b. replace \'none\' with \'\'\n    c. casting to str\n        -> lower\n    d. cleansing\n        -> JobTitle\n            - translate\n            - lower\n        -> FunctionPositionName\n            - translate\n            - lower\n        -> MajorName\n            - translate\n            - lower\n        -> Description\n            - clean text\n            - maintain alphabet\n            - remove single\n            - remove more space\n            - translate\n                -> lower\n            - stemm\n            - tokenize\n            - remove slangwords\n            - remove stopwords\n            - join\n        -> Requirement\n          

In [100]:
stopwords = [i[0] for i in pd.read_csv('data/stopwords.csv').astype(str).values] + ['usia', 'maksimal', 'didik', 'minimal', 'laki', 'perempuan', 'alam', 'milik', 'sd', 'smp', 'sma', 's1', 's2', 's3', 'd1', 'd2', 'd3', 'd4', 'kuasa', 'awas', 'kumpul', 'pria', 'jurus', 'sedia', 'sarjana', 'diploma', 'magister', 'bidang', 'kandidat', 'skill', 'oriented', 'tampil', 'tarik', 'good', 'up', 'paham', 'intermediate', 'baca', 'gambar', 'badan', 'laku', 'laku', 'kena', 'shooting', 'meni', 'buat', 'ipk', 'lulus', 'suka', 'atu', 'hadap', 'penuh', 'jam', 'tara', 'non', 'warna', 'warni', 'smk', 'shift', 'familiar', 'laksana', 'gelar', 'sitac', 'tipu', 'nya', 'ii', 'iii', 'salam', 'to', 'wwm', 'kece', 'kesiap', 'seluru', 'angkat', 'usaha', 'upaya', 'masuk', 'kait', 'syarat', 'susun', 'awat', 'masuk', 'tugas', ]

In [101]:
slangwords = {
    'max': 'maksimal',
    'div': 'divisi',
    'tenant': 'sewa',
    'hokum': 'hukum',
    'branding': 'promosi',
    'stok': 'sedia',
    'termasukaplikasikomputer': 'komputer',
    'telekomunikais': 'telekomunikasi',
    'perinci': 'rinci',
    'risalah': 'surat',
    'mereview': 'ulas',
    'klien': 'langgan',
    'thn': 'tahun',
    'th': 'tahun',
    'min': 'minimal',
    'miniman': 'minimal',
    'jurnalid': 'jurnalis',
    'mayob': 'myob',
    'paja': 'pajak',
    'perban': 'bank',
    'maksimum': 'maksimal',
    'minimum': 'minimal',
    'bhs': 'bahasa',
    'pengorganisasia': 'organisasi',
    'analitis': 'analisis',
    'analisa': 'analisis',
    'memimiliki': 'milik',
    'manejer': 'manajer',
    'menejer': 'manajer',
    'bussines': 'bisnis',
    'development': 'kembang',

    'order': 'pesan',
    'driver': 'supir',
    'mengkoordinir': 'koordinasi',
    'mensupport': 'dukung',
    'padapenyediaan': 'sedia',
    'file': 'berkas',
    'mesan': 'pesan',
    'base': 'dasar',
    'maximum': 'maksimal',
    'accounting': 'akuntansi',
    'system': 'sistem',
    'building': 'bangun',
    'maintenance': 'pelihara',
    'team': 'tim',
    'deadline': 'tenggat',
    'vehicle': 'kendara',
    'networking': 'jaring',
    'installation': 'instalasi',
    'trouble': 'masalah',
    'presentation': 'presentasi',
    'customer': 'langgan',
    'satisfaction': 'puas',
    'planning': 'rencana',
    'tax': 'pajak',
    'mobile': 'seluler',
    'fast': 'cepat',
    'solving': 'pecah',
    'decision': 'putus',
    'making': 'buat',
    'marketing': 'pasar',
    'communication': 'komunikasi',
    'negotiation': 'negosiasi',
    'problem': 'masalah',
    'leasing': 'sewa',
    'service': 'layan',
    'leadership': 'pimpin',
    'manager': 'manajemen',
    'auditing': 'periksa',
    'auditor': 'periksa',
    'audit': 'periksa',
    'apartment': 'apartemen',
    'engineering': 'teknik',
    'chief': 'pimpin',
    'general': 'umum',
    'research': 'teliti',
    'trend': 'cenderung',
    'cash': 'uang',
    'flow': 'alir',
    'payment': 'bayar',
    'supplier': 'pasok',
    'finance': 'uang',
    'journal': 'jurnal',
    'standing': 'duduk',
    'charge': 'biaya',
    'cleaning': 'bersih',
    'lift': 'angkat',
    'adjustment': 'atur',
    'correction': 'koreksi',
    'schedule': 'jadwal',
    'cust': 'langgan',


    'memposting': 'unggah',
    'utilitas': 'guna',
    'ms': 'microsoft',
    'perbankan': 'bank',
    'spv': 'supervisor',
    'komunikatif': 'komunikasi',
    'perencanaan': 'rencana',
    'operasionalisasi': 'operasional',
    'diatahkan': 'arah',
}

In [116]:
stemmer.stem('menyelenggarakan')

'selenggara'

In [ ]:
focus_cleansing.apply(lambda x: change_slangwords(slangwords, x)).apply(lambda x: [i for i in x if i not in stopwords]).values[30:45]

Sebelumnya

In [122]:
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

df_job = df_job.applymap(str.lower)
df_job.EducationLevelName = df_job.EducationLevelName.replace('none', '')

translator = Translator(service_urls=['translate.googleapis.com'])

df_job.JobTitle = df_job.JobTitle.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.FunctionPositionName = df_job.FunctionPositionName.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.MajorName = df_job.MajorName.apply(lambda x: translator.translate(x, dest='id').text.lower())

df_job.Description = df_job.Description.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.Requirement = df_job.Requirement.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())

In [124]:
df_job['JobTitle'] = df_job.JobTitle.str.cat(
    df_job.FunctionPositionName.str.cat(
        df_job.MajorName
    , sep=' ')
, sep=' ').apply(lambda x: ' '.join(set(x.split(' ')))).map(remove_standalonesymbols).map(remove_parenthesesnumber).map(remove_morespace)

df_job.drop(columns=['FunctionPositionName', 'MajorName'], inplace=True)

In [101]:
for i,j in df_job[['Translated', 'JobTitle']].values:
    print(i, '|', j)

direksi sekretaris | direksi sekretaris
staf desain grafis | design desain staff graphic grafis
toko jurusan semua staf awak kapal | jurusan staff crew semua outlet
toko jurusan sma segala awak kapal Pengelola | jurusan sma crew segala outlet manager
pengawas akuntansi | accounting supervisor akuntansi
hukum Pengelola | hukum manager legal
umum qc lapangan sipil operasi teknik | general qc operation lapangan sipil teknik
pajak pengawas akuntansi | pajak accounting supervisor akuntansi
riset jurusan semua staf perkembangan bisnis | research jurusan staff semua business & development
umum jurusan perselingkuhan semua staf | general jurusan staff semua affair
keuangan staf akuntansi  | staff akuntansi & accounting finance
montir listrik jurusan bm-saya semua spv pemeliharaan | bm-me maintenance jurusan semua spv electician 2
Properti jurusan semua pemasaran mendukung | support property jurusan semua marketing
staf informatika dia teknik | informatika staff it teknik
direksi sekretaris sem

In [97]:
df_job.isna().sum()

JobTitle              0
EducationLevelName    0
CityName              0
ProvinceName          0
Description           0
Requirement           0
Translated            0
dtype: int64